MIC prediction for streptomycin

In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib

In [2]:
# Load kmers
data = np.load('../data/interim/streptomycin/kmers/kmer_matrix.npz')
kmers = data['kmers']
kmer_order = data['kmer_order']
kmers.shape

(1406, 1922478)

In [3]:
# Load categories
mics = joblib.load("../data/interim/str_mic_class_dataframe.pkl")
mic_order = joblib.load("../data/interim/str_mic_class_order_dict.pkl")

In [4]:
# Filter kmers for genomes with MIC values
mics = mics.dropna(axis=0, how='any')
mics = mics.loc[mics.streptomycin != 'invalid','streptomycin']
has_mic = np.in1d(kmer_order, mics.index.values)

kmer_order = kmer_order[has_mic]
kmers = kmers[has_mic,:]

In [5]:
kmers.shape

(748, 1922478)

In [6]:
# Encode prediction data
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(mics)
y = le.transform(mics)
X = kmers

In [7]:
# Test / train set
in_training = np.random.rand(len(y)) < 0.8
X_train = X[in_training]
X_test = X[~in_training]
y_train = y[in_training]
y_test = y[~in_training]

In [8]:
# Baseline - uniform likelihood distribution 
from sklearn.metrics import log_loss
nclasses = 7
nsamples = len(y_test)
baseline_pred = np.ones((nsamples, nclasses)) * 1/nclasses
print(log_loss(y_test,baseline_pred))

1.9459101490553135


In [55]:
# Pipelines
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC


fs = SelectPercentile(chi2, percentile=1)
fs.fit(X_train, y_train)
X_train_fs1 = fs.transform(X_train)
X_test_fs1 = fs.transform(X_test)

# fs2 = SelectKBest(mutual_info_classif, k=20000)
# fs2.fit(X_train, y_train)
# X_train_fs2 = fs2.transform(X_train)
# X_test_fs2 = fs2.transform(X_test)

# lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
# model = SelectFromModel(lsvc, prefit=True)
# X_train_mod = model.transform(X_train)
# X_train_mod.shape


In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

#l = LogisticRegression()
#r = RandomForestClassifier()
s = LinearSVC(C=1, penalty='l2')



# y_pred = s.predict_proba(X_test_mod)
# print(log_loss(y_test,y_pred))

In [59]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
s.fit(X_train_fs1,y_train)
y_pred2 = s.predict(X_test_fs1)
print(accuracy_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))

0.5379310344827586
             precision    recall  f1-score   support

          0       0.80      0.33      0.47        12
          1       0.00      0.00      0.00         3
          2       0.36      0.80      0.50         5
          3       0.51      0.44      0.47        50
          4       0.52      0.87      0.65        15
          5       0.00      0.00      0.00         4
          6       0.58      0.62      0.60        56

avg / total       0.53      0.54      0.52       145

[[ 4  0  0  1  6  0  1]
 [ 0  0  0  1  2  0  0]
 [ 0  0  4  0  1  0  0]
 [ 0  1  1 22  2  0 24]
 [ 1  0  1  0 13  0  0]
 [ 0  0  4  0  0  0  0]
 [ 0  0  1 19  1  0 35]]


/home/matt/anaconda3/envs/phenores4/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
le.classes_

array(['16.0000', '32.0000', '4.0000', '64.0000', '8.0000', '<=2.0000',
       '>64.0000'], dtype=object)

In [52]:
from xgboost import XGBClassifier
x = XGBClassifier()
x.fit(X_train_fs2, y_train)
y_pred2 = x.predict(X_test_fs2)
print(accuracy_score(y_test,y_pred2))
print(classification_report(y_test,y_pred2))
cm = confusion_matrix(y_test,y_pred2)
sum(np.diag(cm))/nsamples
print(cm)

0.5172413793103449
             precision    recall  f1-score   support

          0       0.20      0.08      0.12        12
          1       0.00      0.00      0.00         3
          2       0.33      0.40      0.36         5
          3       0.51      0.50      0.51        50
          4       0.44      0.73      0.55        15
          5       0.33      0.25      0.29         4
          6       0.61      0.62      0.62        56

avg / total       0.50      0.52      0.50       145

[[ 1  0  0  3  6  0  2]
 [ 0  0  0  1  2  0  0]
 [ 0  0  2  0  2  1  0]
 [ 2  0  0 25  2  1 20]
 [ 2  0  0  2 11  0  0]
 [ 0  0  3  0  0  1  0]
 [ 0  0  1 18  2  0 35]]


/home/matt/anaconda3/envs/phenores4/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/matt/anaconda3/envs/phenores4/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [54]:
cm = confusion_matrix(y_test,y_pred2)
(sum(np.diag(cm))+18+20)/nsamples


0.7793103448275862

In [42]:
le.classes_

array(['16.0000', '32.0000', '4.0000', '64.0000', '8.0000', '<=2.0000',
       '>64.0000'], dtype=object)